### Preference_Similarity_model

In [ ]:
# -*- coding: utf-8 -*-
from nltk.tokenize import RegexpTokenizer
import nltk
import jieba
import jieba.posseg as pseg
import jieba.analyse
import codecs
from nltk.corpus import stopwords 
import re
import pandas as pd
import numpy as np
import scipy.stats
import scipy as sp
from scipy.stats import entropy
from scipy.spatial import distance
from numpy.linalg import norm
import datetime
import gensim

#### 全域變數

In [ ]:
#global variable
#4大模組
Prm = 0
Pic = 0
Pop = 0
Act = 0
#activate module
#用戶發布的post數量
Ap = 0
#用戶最近一個月的post數量
Apm = 0
#用戶最新一次po文在多久以前
Apt = None
#用戶追蹤了多少人
Af = 0

In [ ]:
#ig與fb 圖片的相似度分數
ig_fb_picture_similarity_score_list = []

In [ ]:
 #去除空值row
oringnal_df = oringnal_df.dropna(subset=["message"])
oringnal_ig_df = oringnal_ig_df.dropna(subset=["post content"])
oringnal_df['new'] = "NEW"
oringnal_ig_df['new content'] = "None"

#### 文字前處理

In [1]:
# 移除停留詞
file_name = 'dictionary/stopword.txt'
with open(file_name,'r',encoding="utf-8") as f:
    stop_words = f.readlines()
def cut_words(text):
    #斷詞 精準模式
    text = jieba.cut(text, cut_all=False)
    return text  
def remove_stop_words(file_name,seg_list):
    new_list = []
    English_re = re.compile(r'[A-Za-z]')
    Number_re = re.compile(r'[0-9]')
    emoji_re = re.compile("[\u4e00-\u9FFF]")
    global stop_words
    stop_words = [stop_word.rstrip() for stop_word in stop_words]
    for seg in seg_list:
        if seg not in stop_words:
            #去除含有數字、英文、空格的元素
            if (seg.isdigit() is True or seg is ' ' or bool(re.match(English_re, seg)) is True or bool(re.match(Number_re, seg)) is True or bool(re.match(emoji_re, seg)) is False):
                continue
            else:
                #關鍵詞，去除停用字 (提高準確度!!!!!!!)
                seg = jieba.analyse.extract_tags(seg, topK=20, withWeight=False, allowPOS=())
#                 print(' '.join(seg))
                new_list.append(' '.join(seg))
            #if bool(re.match(my_re,seg)) is False and seg.isdigit() is False and seg is not ' ': #去除英文元素
                #new_list.append(seg) #若在for loop裡用remove的話則會改變總長度
    return new_list

In [ ]:
#臉書content 斷字斷詞、清理
oringnal_df['message'] = oringnal_df['message'].apply(lambda text : cut_words(text))
oringnal_df['new'] = oringnal_df['message'].apply(lambda text : remove_stop_words(file_name,text))

In [ ]:
#只保留非hashtags的content內容
def remain_instagram_post(text):
    a = re.findall(r"#(\w+)",text)
    b = re.findall(r"(\w+)",text)
    c = list(set(b) - set(a))
    return c

In [ ]:
#備份content 與 呼叫remain_instagram_post動作
oringnal_ig_df['post content copy'] = oringnal_ig_df['post content']
oringnal_ig_df['post content'] = oringnal_ig_df['post content'].apply(lambda text : remain_instagram_post(text))

In [ ]:
#將content陣列轉回字串型式
oringnal_ig_df['post content'] = oringnal_ig_df['post content'].apply(lambda text : ' '.join(text))

In [ ]:
#instagram 斷字斷詞、清理
oringnal_ig_df['post content'] = oringnal_ig_df['post content'].apply(lambda text : cut_words(text))
oringnal_ig_df['new content'] = oringnal_ig_df['post content'].apply(lambda text : remove_stop_words(file_name,text))

In [ ]:
#oringnal_ig_df['new content']

In [ ]:
#將oringnal_df['new'] form list to string
oringnal_df['new_copy'] = oringnal_df['new']
oringnal_df['new'] = oringnal_df['new'].apply(lambda text : ' '.join(text))
#將oringnal_ig_df['new content'] form list to string
oringnal_ig_df['new content'] = oringnal_ig_df['new content'].apply(lambda text : ' '.join(text))

In [ ]:
# print(oringnal_df.head(10))

In [ ]:
#update the index from 0 to finall
oringnal_df.index = range(len(oringnal_df))
oringnal_ig_df.index = range(len(oringnal_ig_df))

In [ ]:
# print(len(oringnal_df),len(oringnal_ig_df))

In [ ]:
#要將兩份文檔維度調整成一樣
if len(oringnal_df)>len(oringnal_ig_df):
    oringnal_df.drop(oringnal_df.index[len(oringnal_ig_df):], inplace=True,axis = 0)
else:
    oringnal_ig_df.drop(oringnal_ig_df.index[len(oringnal_df):], inplace=True,axis = 0)

#### 進行LDA模型訓練

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
#後續要處理的文字部分存成corpus
corpus=oringnal_df["new"]
ig_corpus=oringnal_ig_df["new content"]

In [ ]:
cntVector = CountVectorizer(analyzer='word',token_pattern=u"(?u)\\b\\w+\\b")
ig_cntVector = CountVectorizer(analyzer='word',token_pattern=u"(?u)\\b\\w+\\b")
cntTf = cntVector.fit_transform(corpus)
ig_cntTf = ig_cntVector.fit_transform(ig_corpus)
# print (cntTf)
# print (ig_cntTf)

In [ ]:
n_topics = 10
lda = LatentDirichletAllocation(n_components = n_topics, 
                                max_iter=50,
                                learning_method='batch')
docres = lda.fit_transform(cntTf) 
ig_docres = lda.fit_transform(ig_cntTf) 

In [ ]:
# print (lda.components_)

In [ ]:
def jensen_shannon_distance(p, q):
    global Prm 
    """
    method to compute the Jenson-Shannon Distance 
    between two probability distributions
    """

    # convert the vectors into numpy arrays in case that they aren't
    p = np.array(p)
    q = np.array(q)

    # calculate m
    m = (p + q) / 2

    # compute Jensen Shannon Divergence
    divergence = (scipy.stats.entropy(p, m) + scipy.stats.entropy(q, m)) / 2

    # compute the Jensen Shannon Distance
    distance = np.sqrt(divergence)
    around = np.around(distance, decimals=4)
    Prm = around.max()
    
    
    return Prm

#### *得到 Prm 分數

In [ ]:
Prm = jensen_shannon_distance(docres,ig_docres)
print("Prm : ",Prm)

In [ ]:
# def print_top_words(model, feature_names, n_top_words):
#     #打印每个主题下权重较高的term
#     for topic_idx, topic in enumerate(model.components_):
#         try:
#             print ("Topic #%d:" % topic_idx)
#             print (" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
#         except Exception as e: print(e)

In [ ]:
# n_top_words=5
# tf_feature_names = cntVector.get_feature_names()
# print(tf_feature_names)
# print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
# n_top_words=50
# tf_feature_names = ig_cntVector.get_feature_names()
# print_top_words(lda, tf_feature_names, n_top_words)

### Popularity calculation module

In [ ]:
#instagram 斷字斷詞、清理
# oringnal_ig_df['leavemessage content'] = oringnal_ig_df['leavemessage content'].apply(lambda text : cut_words(text))
# oringnal_ig_df['leavemessage content']
oringnal_ig_df['leavemessage content'] = oringnal_ig_df['leavemessage content'].astype(str)
oringnal_ig_df['leavemessage content'] = oringnal_ig_df['leavemessage content'].apply(lambda text : cut_words(text))
oringnal_ig_df['new leavemessage content'] = oringnal_ig_df['leavemessage content'].apply(lambda text : remove_stop_words(file_name,text))

In [ ]:
ig_leavecontent_sentence = oringnal_ig_df['new leavemessage content']

#### 進行SnowNLP

In [ ]:
from snownlp import SnowNLP
#越接近1表示正面情緒
#越接近0表示負面情緒

In [ ]:
#將oringnal_ig_df['new leavemessage content'] form list to string
oringnal_ig_df['new leavemessage content'] = oringnal_ig_df['new leavemessage content'].apply(lambda text : ' '.join(text))
ig_leavemessage_content = oringnal_ig_df['new leavemessage content']

In [ ]:
# ig_leavemessage_content

In [ ]:
sentence_score = 0
sentence_count = 0
for text in ig_leavemessage_content:
    #print(text)
    sentence_count+=1
    try:
        s = SnowNLP(text)
    except Exception as e: pass

    sentence_score += s.sentiments
if sentence_score != 0:
    ig_leavemessage_sentiments = sentence_score/sentence_count
else:
    print("sentence_score",sentence_score)

In [ ]:
ig_content_likes = oringnal_ig_df['num of content likes'].mean()
ig_content_likes = np.around(ig_content_likes,decimals=2)
#print(ig_content_likes)

In [ ]:
# print(oringnal_ig_df['avg response'][0])
# print(oringnal_ig_df['num of followers'][0])
# print(oringnal_ig_df['num of tags'][0])

#### 進行正歸化到區間 0-1

In [ ]:
# Pop_array = [[ig_leavemessage_sentiments,ig_content_likes,oringnal_ig_df['avg response'][0],oringnal_ig_df['num of followers'][0],oringnal_ig_df['num of tags'][0]]]
Pop_array = [[ig_leavemessage_sentiments],[ig_content_likes],[oringnal_ig_df['avg response'][0]],[oringnal_ig_df['num of followers'][0]],[oringnal_ig_df['num of tags'][0]]]
#print(Pop_array)
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler(copy=True, feature_range=(0, 1))
x_minmax = min_max_scaler.fit_transform(Pop_array)

#### 得到Pop分數

In [ ]:
Pop = x_minmax.mean()
print("Pop : " , Pop)

### Activity Calculation  Module 

In [ ]:
from datetime import datetime

# 用戶發布的post數量。
Ap = oringnal_ig_df['num of posts'][0]
# 用戶最近一個月的post數量。
Apm = oringnal_ig_df['one month posts'][0]
# 用戶最新一次po文在多久以前。
oringnal_ig_df['post_time'][0] = datetime.strptime(oringnal_ig_df['post_time'][0], "%Y/%m/%d")
# print(type(oringnal_ig_df['post_time'][0]))
# 用戶追蹤了多少人。
Af = oringnal_ig_df['num of follows'][0]

#### 進行正歸化到區間 0-1

In [ ]:
import datetime
# 用戶最新一次po文在多久以前。由於是時間要素所以利用線性回歸方程式來正規化
#現在時間
now = datetime.datetime.now()
if (now - oringnal_ig_df['post_time'][0]).days>10:
    Apt = 0
elif 0<=(now - oringnal_ig_df['post_time'][0]).days<=10:
    Apt = -0.1*(now - oringnal_ig_df['post_time'][0]).days+1

Pop_array = [[Ap],[Apm],[Af]]
x_minmax = min_max_scaler.fit_transform(Pop_array)
for number in x_minmax:
    Act += float(number)
Act += Apt
Act /= 4
print("Act : " , Act)

### Picture similarity module

#### 找出每篇instagram post 的hashtags

In [ ]:
#oringnal_ig_df['post content copy']

In [ ]:
#每篇post hashtags的部分
oringnal_ig_df['post content copy'] = oringnal_ig_df['post content copy'].apply(lambda text : re.findall(r"#(\w+)",text))

In [ ]:
#print(oringnal_ig_df['post content copy'])

In [ ]:
# oringnal_df['new'] = oringnal_df['new'].apply(lambda text : list(text))
#oringnal_df['new_copy']

In [ ]:
oringnal_ig_df['post content copy'] = oringnal_ig_df['post content copy'].apply(lambda text : ' '.join(text))

In [ ]:
#臉書content 斷字斷詞、清理
oringnal_ig_df['post content copy'] = oringnal_ig_df['post content copy'].apply(lambda text : cut_words(text))
oringnal_ig_df['post content copy_stopwords'] = oringnal_ig_df['post content copy'].apply(lambda text : remove_stop_words(file_name,text))

In [ ]:
#print(oringnal_ig_df['post content copy_stopwords'])

#### fb post 利用TextRank找出top 3關鍵字

In [ ]:
#將陣列轉成str
oringnal_df['new_copy'] = oringnal_df['new_copy'].apply(lambda text : ' '.join(text))
# 將臉書content 透過textrank 只保留分數最高的三個詞，且不返回權重值
oringnal_df['new_copy'] = oringnal_df['new_copy'].apply(lambda text : jieba.analyse.textrank(text, topK=3, withWeight=False))

In [ ]:
#導入Word2vec模組
from gensim.models import Word2Vec
model_word2vec = Word2Vec.load("./word2vec-tutorial-master/word2vec.model")

#### 進行word2vec & cosine similarity

In [ ]:
#進行word2vec 回傳單詞向量
def word2vec(word):
    global model_word2vec
    try:
        model_word2vec[word]
        return model_word2vec[word]
    except Exception as e: pass
    
#計算cosine similarity
def cosdis(v1, v2):
    try:
        cosine_similarity = np.dot(v1, v2)/(np.linalg.norm(v1)* np.linalg.norm(v2))
        return cosine_similarity.item()
    except Exception as e: pass

In [ ]:
def calculate_picture_similarity(list_A,list_B):
    global ig_fb_picture_similarity_score_list
    
    a = 0.0
    threshold = 0.80     # if needed
    for key in list_A:
        for word in list_B:
            try:
                res = cosdis(word2vec(word), word2vec(key))
                #從所有word2vec 從所有pair simlarity中取最大的score
                try:
                    if res >= a:
                        a = res
                except Exception as e: pass
#                 print("The cosine similarity between : {} and : {} is: {}".format(word, key, res*100))
#                 if res > threshold:
#                 print("Found a word with cosine distance > 80 : {} with original word: {}".format(word, key))
            except IndexError:
                pass
    ig_fb_picture_similarity_score_list.append(a)
    return ig_fb_picture_similarity_score_list

In [ ]:
if len(oringnal_ig_df['post content copy_stopwords'])>= len(oringnal_df['new_copy']):
    for i in range(len(oringnal_ig_df['post content copy_stopwords'])):
        list_A = oringnal_ig_df['post content copy_stopwords'][i]
        list_B = oringnal_df['new_copy'][i]
        calculate_picture_similarity(list_A,list_B)

In [ ]:
Pic = sum(ig_fb_picture_similarity_score_list)/len(ig_fb_picture_similarity_score_list)
print("Pic : ",Pic)

In [ ]:
# if model_choice == 0:
#     print("model 0")
#     all_score = Prm*0.926 + Pic*0.926 + Pop*10.4 + Act*1.04
# elif model_choice == 1:
#     print("model 1")
#     all_score = Pic + Pop + Act
# elif model_choice == 2:
#     print("model 2")
#     all_score = Prm + Pop + Act
# elif model_choice == 3:
#     print("model 3")
#     all_score = Prm + Pic + Act
# else:
#     print("model 4")
#     all_score = Prm + Pic + Pop
# print("all_score " , all_score)